# The purpose of this notebook is to analyze and make predictions using chest compression data

In [ ]:
import pandas as pd
import numpy as np

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

from scipy import signal

In [ ]:
# Load data
df_mat = pd.read_csv('data/air_mat_comp_600.csv')
print(df_mat.shape)
dfg_pil = pd.read_csv('data/Pillow-CPR.csv')
print(dfg_pil.shape)
dfg_ted = pd.read_csv('data/TeddyBear-CPR.csv')
print(dfg_ted.shape)
dfr_pil = pd.read_csv('data/pillow_reed.csv')
print(dfr_pil.shape)
dfr_ted = pd.read_csv('data/teddy_reed.csv')
print(dfr_ted.shape)

## Absolute truth: Number of compressions per data set
- Mattress ------> 600 (30 per set)
- Pillow George -> 150 (30 per set)
- Pillow Reed ---> 150?
- Teddy George --> 150 (30 per set)
- Teddy Reed ----> 150?

### Add labels to each of the dataframes to indicate their compression object

In [ ]:
df_mat['object'] = "mattress"
dfg_pil['object'] = "pillow_george"
dfg_ted['object'] = "teddy_george"
dfr_pil['object'] = "pillow_reed"
dfr_ted['object'] = "teddy_reed"

In [ ]:
# Rename columns in df_mat
df_mat = df_mat.rename(columns={'Linear Acceleration x (m/s^2)': 'Acceleration x (m/s^2)',\
                       'Linear Acceleration y (m/s^2)': 'Acceleration y (m/s^2)',\
                       'Linear Acceleration z (m/s^2)':'Acceleration z (m/s^2)'}) 

### Create time delta column

In [ ]:
def time_delta(data):
    data['prev_time'] = data['Time (s)'].shift(1)
    data['time_delta'] = data['Time (s)'] - data['prev_time']
    return data

In [ ]:
df_mat = time_delta(df_mat)
dfg_pil = time_delta(dfg_pil)
dfg_ted = time_delta(dfg_ted)
dfr_pil = time_delta(dfr_pil)
dfr_ted = time_delta(dfr_ted)

In [ ]:
dfg_pil.head()

### Stack all dataframes

In [ ]:
print("expected row count:",df_mat.shape[0] + 
                            dfg_pil.shape[0] +
                            dfg_ted.shape[0] +
                            dfr_pil.shape[0] +
                            dfr_ted.shape[0])
df = pd.concat([df_mat,dfg_pil,dfg_ted,dfr_pil,dfr_ted], ignore_index=True,sort=False)
print('actual row count  :',df.shape[0])

In [ ]:
dfr_ted.tail()

In [ ]:
df.info()

### Plot data:
- Are peaks easy to visualize?

In [ ]:
df.columns

In [ ]:
# df[df['object'] == 'mattress'].plot(x='Time (s)', y=['Acceleration x (m/s^2)', 
#                                                      'Acceleration y (m/s^2)',
#                                                      'Acceleration z (m/s^2)', 
#                                                      'Absolute acceleration (m/s^2)'], kind="bar", figsize=(9, 8))

In [ ]:
# Mattress
df[df['object'] == 'mattress'].plot( 'Time (s)' , 'Absolute acceleration (m/s^2)')
#df[df['object'] == 'mattress'].plot( 'Time (s)' , 'Acceleration x (m/s^2)')
#df[df['object'] == 'mattress'].plot( 'Time (s)' , 'Acceleration y (m/s^2)')
#df[df['object'] == 'mattress'].plot( 'Time (s)' , 'Acceleration z (m/s^2)')
plt.title("Mattress Compressions over Time")

In [ ]:
# George pillow
df[df['object'] == 'pillow_george'].plot( 'Time (s)' , 'Absolute acceleration (m/s^2)')
plt.title("Pillow George Compressions over Time")

In [ ]:
# George teddy
df[df['object'] == 'teddy_george'].plot( 'Time (s)' , 'Absolute acceleration (m/s^2)')
plt.title("Teddy George Compressions over Time")

In [ ]:
# Reed pillow
df[df['object'] == 'pillow_reed'].plot( 'Time (s)' , 'Absolute acceleration (m/s^2)')
plt.title("Pillow Reed Compressions over Time")

In [ ]:
# Reed teddy
df[df['object'] == 'teddy_reed'].plot( 'Time (s)' , 'Absolute acceleration (m/s^2)')
plt.title("Teddy Reed Compressions over Time")

## Notes: 
- All compression data seems to be well spaced
- George Teddy data seems to be 90 compressions
    - Confirm with George

### Create the following:
- Half second increment column
  - Ascneding in value
- Rolling std
  - std across half second increment blocks

In [ ]:
df.head()

In [ ]:
# Roughly how many rows in .1 seconds?
df['time_delta'].head(11).sum()

In [ ]:
# A compression may occur in a fraction of a second
# Take the rolling std for every ten rows
df['rolling_stdev'] = df['Absolute acceleration (m/s^2)'].rolling(10).std()

In [ ]:
df.head(20)

In [ ]:
df[df['object'] == 'mattress'].plot( 'Time (s)' , 'rolling_stdev')
plt.title("Mattress: Rolling Standard Dev over Time")

In [ ]:
df[df['object'] == 'pillow_george'].plot( 'Time (s)' , 'rolling_stdev')
plt.title("Pillow George: Rolling Standard Dev over Time")